## Calculating the Trajectory Errors

- how to calculate the trajectory errors and seperate the files into bad and good trajectories

- Currently need to work out how to get python to show the integration errors between the forward and reverse trajectories

In [1]:
# STEP 0: Must be done before anything each time notebook is loaded
# Import PySplit

import pysplit as py
import numpy as np
import pickle as pickle
from pprint import pprint
import os

#### Trajectory generation
- basic trajectory, test used for just this analysis

In [2]:
# STEP 1: TRAJECTORY GENERATION (not yet tested on mac)
# check out bulk_trajen_example.py

#directory setup
working_dir = r'/Volumes/Seagate_Backup/hysplit4/working'
storage_dir = r'/Volumes/Seagate_Backup/hysplit4/pysplitgen/XITEST'
meteo_dir = r'/Volumes/Seagate_Backup/HYSPLIT/ncep_monYYYY'
# meteo_dir = r'C:\hysplit4\gdas'

#file basename
basename = 'XITEST'

if not os.path.exists(storage_dir):
    os.makedirs(storage_dir)

#Arguments
years = [2007]
#years=range(2006,2017)
months = [1]
#months=range(1,13)
hours = [4]
altitudes = [1500]
location = (-10.25, 105.4)
runtime = -168

#Generate trajs
py.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir,
                          years, months, hours, altitudes, location, runtime,
                          monthslice=slice(0, 32, 1), meteo_bookends=([0],[0]), get_reverse=True, get_clipped=True, hysplit="/Volumes/Seagate_Backup/hysplit4/exec/hyts_std")

#### Grouping the trajectories

- determine the distance between the start and end of trajectories, the vector 
- can seperate the rainy status ones (based in Summer values)

In [3]:
# STEP 2: INITIALISE TRAJECTORY GROUP
# Group trajectories and then cycle through a group to determine distance, vector and rain status.
# check out traj_trajgroup_basics_examle.py

# Initialise Trajectories and Create a Trajectory Group
test=[]
test= py.make_trajectorygroup(r'/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/*summer*')
# Calculate distance and vectors
for traj in test:
    traj.calculate_distance()
    traj.calculate_vector()

#### Calculating the integration error

- first the reversetraj has to be loaded, this is because the integration error is the physical and numerical error. This is calculated by the distance between the original trajectory and the reverse trajectory start and end points. 

- then the integration error can be calculated by traj.calculate_integrationerr()

    - this also should inclue te traj.integration_error (the relative error%) and traj.integration_error_abs (the absolute error in meters)

In [4]:
# integration error filtering# integr 
#first load reverse Traj
for traj in test:
    traj.load_reversetraj()
#then calculate integration error
# Values computed when calling ``Trajectory.calculate_integrationerr()``:
#     ``Trajectory.integration_error``, the relative error (%)
#     ``Trajectory.integration_error_abs``, the absolute error (meters)


Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070101041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 4 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070102041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 28 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070103041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 52 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070104041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 76 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070105041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 100 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070106041.0 has a bad

/Users/jess/anaconda3/envs/pysplitenv/lib/python3.6/site-packages/pysplit/hypath.py:144: RuntimeWarning: invalid value encountered in arccos
  np.cos(lon[0] - lon)) * 6371) * 1000


In [5]:
c=0
for traj in test:
    traj.calculate_integrationerr()
    print(c)
    c+=1
    




Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070101041.0
0
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070102041.0
1
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070103041.0
2
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070104041.0
3
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070105041.0
4
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070106041.0
5
Integration error calculation skipped for
                  Trajectory /Volumes/Se

In [6]:
test

The problem with the code below (relative errors and the integration_error code) is that for the trajectories which do not calculate integration erros it still tries to loop through them. 

- solution would be to try and create a trajectory group which excludes the trajectories which do not fit in, and do not have trajectories integration errors.



- how to do this???

In [7]:
badreverse=[] #identigy whihc of the trajectories are bad and then put them into a badreverse list of the trajectory.trajid.
# this is done through .append below

for traj in test:
    try: 
        print(traj.integration_error)
    except:
        print(traj.trajid)
        badreverse.append(traj.trajid)

/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070101041.0
/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070102041.0
/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070103041.0
/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070104041.0
/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070105041.0
/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070106041.0
/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070107041.0
0.024267485942220317
0.16125892999933739
0.0341340389544233
0.010168053088224056
0.0013476617180504973
0.004062116443966302
0.013115733808061752
0.0016123889109760077
0.0026486099207265767
0.013168370352858831
0.06271238720687736
0.015155761455574708
0.024314119941599807
0.14834750635356445
0.0633505304549389
0.04238537533324472
0.006382437522313771
0.0017869113397068587
0.006563775608023851
0.005920599275696525
0.006172886817259317
0.0

In [8]:
badreverse


['/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070101041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070102041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070103041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070104041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070105041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070106041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070107041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testfeb1100summer20070201041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testfeb1100summer20070202041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testfeb1100summer20070203041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testfeb1100summer20070204041.0',
 '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testfeb1100summer20070205

In [9]:
# .pop is used to exclude a number of trajectories from the list, based on a list created previously., i.e the badreverse
#list of trajectories can be used. 

test.pop(trajid=badreverse) 
print('Result: ', test.trajcount, 'trajectories')

Result:  69 trajectories


In [10]:
relative_errors = [traj.integration_error for traj in test]
cutoff = np.mean(relative_errors) + (np.std(relative_errors) * 2)

print('Integration error upper limit: ', cutoff)

Integration error upper limit:  0.11975214324557325


In [11]:
bad = []
for traj in test:
    if traj.integration_error > cutoff:
        bad.append(traj.trajid)

print('Expectation: ', test.trajcount, 'trajectories -', len(bad),
      'bad trajectories =', test.trajcount-len(bad), 'trajectories')

Expectation:  69 trajectories - 7 bad trajectories = 62 trajectories


In [12]:
test.pop(trajid=bad)

print('Result: ', test.trajcount, 'trajectories')


Result:  62 trajectories


### Testing for if the trajectories contain rainfall days or not

- create a dry and rainy trajectories group
- creat/identify the moisture uptake for the different trajectories


In [16]:
#appending the rainy day trajectories
rainylist = []

for traj in test:
    traj.set_rainstatus()
    if traj.rainy:
        rainylist.append(traj)
        
rainy_test = py.TrajectoryGroup(rainylist)
dry_test = test - rainy_test
print('Total trajectories in total group: ',test.trajcount)
print('Total trajectories in dry group: ',dry_test.trajcount)
print('Total trajectories in rainy group: ',rainy_test.trajcount)
pprint(vars(rainy_test))

Total trajectories in total group:  62
Total trajectories in dry group:  31
Total trajectories in rainy group:  31
{'trajcount': 31,
 'trajectories': [<pysplit.traj.Trajectory object at 0x12409b358>,
 'trajids': ['/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070110041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070111041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070114041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070117041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070118041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070120041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070122041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070123041.0',
             '/Volumes/Seagate_B

In [17]:
# moisture uptake identification of the moisture source regions for the rainy day trajectories
moisture = []
for traj in rainy_test:
    traj.moisture_uptake(precipitation=-0.2,evaporation=0.2,interval=3)
    moisture.append(traj)
    

In [19]:
vars(moisture[2])

{'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
 Timestep                                                                  
  0.0           0.0     891.4       0.1         630.2               67.2   
 -1.0          -1.0     890.1       0.1         630.2               68.0   
 -2.0          -2.0     888.7       0.1         630.2               68.8   
 -3.0          -3.0     887.1       0.1         630.2               69.5   
 -4.0          -4.0     885.3       0.1         630.2               70.1   
 -5.0          -5.0     883.3       0.1         681.1               69.0   
 -6.0          -6.0     881.7       0.1         731.9               68.1   
 -7.0          -7.0     880.3       0.0         782.8               67.3   
 -8.0          -8.0     879.1       0.0         833.6               66.7   
 -9.0          -9.0     878.2       0.0         884.5               66.2   
 -10.0        -10.0     877.7       0.0         935.3               66.0   
 -11